In [46]:
import logging
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.magma.compiler.subcompilers.nc.ncproc_compiler import CompilerOptions
from lava.utils.profiler import Profiler

np.set_printoptions(linewidth=110)  # Increase the line lenght of output cells.

## Version 1: running without transformation to spikes (leads to kernel dying)

In [2]:
def load_npz_frames(file_name: str) -> np.ndarray:
    '''
    :param file_name: path of the npz file that saves the frames
    :type file_name: str
    :return: frames
    :rtype: np.ndarray
    '''
    return np.load(file_name, allow_pickle=True)['frames'].astype(np.float32)

In [10]:
def CIFAR_DVS_dataset():
    dataset =  datasets.DatasetFolder('cifar_project/CIFAR/frames_number_10_split_by_number', loader = load_npz_frames, extensions=('.npz', '.npy'))
    training_set, testing_validation_set = split_to_train_test_set(0.995, dataset, 10)
    #testing_validation_set = torch.load("cifar_dvs_dataset_mini_mini_reshaped.pt")
    return testing_validation_set

In [7]:
import torch

In [11]:
cifar_data_reshaped = CIFAR_DVS_dataset()

data loaded


In [24]:
import os
trained_weights_path = os.path.join('.', 'network.npy')

real_path_trained_wgts = os.path.realpath(trained_weights_path)

wb_list = np.load(real_path_trained_wgts, encoding='latin1', allow_pickle=True).item()
w0 = wb_list['blocks.0.synapse.weight_v'].astype(np.int32)
w0_shape = np.shape(w0)
w0 = w0.reshape((w0_shape[0], w0_shape[1]))
w1 = wb_list['blocks.1.synapse.weight_v'].astype(np.int32)
w1_shape = np.shape(w1)
w1 = w1.reshape((w1_shape[0], w1_shape[1]))
w2 = wb_list['blocks.2.synapse.weight_v'].astype(np.int32)
w2_shape = np.shape(w2)
w2 = w2.reshape((w2_shape[0], w2_shape[1]))
u0 = wb_list['blocks.0.neuron.current_decay'].astype(np.int32)
u1 = wb_list['blocks.1.neuron.current_decay'].astype(np.int32)
u2 = wb_list['blocks.2.neuron.current_decay'].astype(np.int32)

v0 = wb_list['blocks.0.neuron.voltage_decay'].astype(np.int32) 
v1 = wb_list['blocks.1.neuron.voltage_decay'].astype(np.int32) 
v2 = wb_list['blocks.2.neuron.voltage_decay'].astype(np.int32) 

In [ ]:
from lava.proc.io.dataloader import SpikeDataloader 
lif_src = SpikeDataloader(dataset = cifar_data_reshaped)
dense = Dense(weights=w0)
lif_dest = LIF(shape=(512,), vth = 1.25, dv = v0, du = u0)
dense1 = Dense(weights=w1)
lif_dest_1 = LIF(shape=(512,), vth = 1.25, dv = v1, du = u1)   
dense2 = Dense(weights=w2)
lif_dest_2 = LIF(shape=(10,), vth = 1.25, dv = v2, du = u2)        
# Connect the OutPort of lif_src to the InPort of dense.
lif_src.s_out.connect(dense.s_in)
dense.a_out.connect(lif_dest.a_in)
lif_dest.s_out.connect(dense1.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense1.a_out.connect(lif_dest_1.a_in)
lif_dest_1.s_out.connect(dense2.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense2.a_out.connect(lif_dest_2.a_in)

## Version 2: Transformation into spikes (leads to source destination error)

In [44]:
def toSpikeTensor(event, emptyTensor, samplingTime=1, randomShift=False, binningMode='OR'):
  event_dim = 2
  if randomShift is True:
    tSt = np.random.randint(
      max(
        int(event['t'].min() / samplingTime),
        int(event['t'].max() / samplingTime) - emptyTensor.shape[3],
        emptyTensor.shape[3] - int(event['t'].max() / samplingTime),
        1,
      )
    )
  else:
    tSt = 0

  xEvent = np.round(event['x']).astype(int)
  pEvent = np.round(event['p']).astype(int)
  tEvent = np.round(event['t']/samplingTime).astype(int) - tSt

  if event_dim == 1:
    validInd = np.argwhere((xEvent < emptyTensor.shape[2]) &
                  (pEvent < emptyTensor.shape[0]) &
                  (tEvent < emptyTensor.shape[3]) &
                  (xEvent >= 0) &
                  (pEvent >= 0) &
                  (tEvent >= 0))
    if binningMode.upper() == 'OR':
      emptyTensor[pEvent[validInd],
            0,
            xEvent[validInd],
            tEvent[validInd]] = 1/samplingTime
    elif binningMode.upper() == 'SUM':
      emptyTensor[pEvent[validInd],
            0,
            xEvent[validInd],
            tEvent[validInd]] += 1/samplingTime
    else:
      raise Exception('Unsupported binningMode. It was {}'.format(binningMode))

  elif  event_dim == 2:
    yEvent = np.round(event['y']).astype(int)
    validInd = np.argwhere((xEvent < emptyTensor.shape[2]) &
                  (yEvent < emptyTensor.shape[1]) &
                  (pEvent < emptyTensor.shape[0]) &
                  (tEvent < emptyTensor.shape[3]) &
                  (xEvent >= 0) &
                  (yEvent >= 0) &
                  (pEvent >= 0) &
                  (tEvent >= 0))

    if binningMode.upper() == 'OR':
      emptyTensor[pEvent[validInd],
            yEvent[validInd],
            xEvent[validInd],
            tEvent[validInd]] = 1/samplingTime
    elif binningMode.upper() == 'SUM':
      emptyTensor[pEvent[validInd],
            yEvent[validInd],
            xEvent[validInd],
            tEvent[validInd]] += 1/samplingTime
    else:
      raise Exception('Unsupported binningMode. It was {}'.format(binningMode))

  return emptyTensor



def toSpikeArray(event, samplingTime=1, dim=None):
  event_dim = 1
  if 'y' in event.keys():
			event_dim = 2
  if event_dim == 1:
    if dim is None: dim = ( np.round(max(event['p'])+1).astype(int),
                np.round(max(event['x'])+1).astype(int),
                np.round(max(event['t'])/samplingTime+1).astype(int) )
    frame = np.zeros((dim[0], 1, dim[1], dim[2]))
  elif event_dim == 2:
    if dim is None: dim = ( np.round(max(event['p'])+1).astype(int),
                np.round(max(event['y'])+1).astype(int),
                np.round(max(event['x'])+1).astype(int),
                np.round(max(event['t'])/samplingTime+1).astype(int) )
    # using 200 here instead of dim[3]
    frame = np.zeros((dim[0], dim[1], dim[2], dim[3]))
  return toSpikeTensor(event, frame, samplingTime).reshape(dim)

In [45]:
import os
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F
import numpy as np

class SpikeDataset(Dataset):
    def __init__(self, root_folder, transform=None):
        self.root_folder = root_folder
        self.transform = transform
        self.classes, self.class_to_idx = self._find_classes()
        self.samples = self._make_dataset()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, target = self.samples[idx]
        data = np.load(file_path)
        arrays = {key: data[key] for key in data.keys()}
        spike = toSpikeArray(arrays,samplingTime = 100000)
        spike_reshaped = np.moveaxis(spike, 1, -1)
        spike_reshaped = np.moveaxis(spike_reshaped, 1, -1)
        # Perform average pooling only along the last two dimensions (20x20)
        spike_pooled = F.avg_pool2d(torch.tensor(spike_reshaped), kernel_size=7, stride=7, padding=0).numpy()

        # Reshape back to the original shape
        spike_pooled = np.moveaxis(spike_pooled, -1, 1)
        spike_pooled = np.moveaxis(spike_pooled, -1, 1)
        
        return spike_pooled.reshape(-1, np.shape(spike_pooled)[3]), target

    def _find_classes(self):
        # Find the class folders in your dataset
        classes = [d for d in os.listdir(self.root_folder) if os.path.isdir(os.path.join(self.root_folder, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def _make_dataset(self):
        # Create a list of file paths and their corresponding labels
        samples = []
        for target_class in self.classes:
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(self.root_folder, target_class)
            for root, _, fnames in sorted(os.walk(target_dir)):
                for fname in fnames:
                    path = os.path.join(root, fname)
                    item = (path, class_index)
                    samples.append(item)
        return samples

root_folder = "events_np/content/events_np/content/CIFAR/events_np"
spike_dataset = SpikeDataset(root_folder)


In [51]:
from lava.proc.io.dataloader import SpikeDataloader, PySpikeModelFloat
lif_src = SpikeDataloader(dataset = spike_dataset)
dense = Dense(weights=np.ones((512, 648)))
lif_dest = LIF(shape=(512,))
dense1 = Dense(weights=np.ones((512, 512)))
lif_dest_1 = LIF(shape=(512,))   
dense2 = Dense(weights=np.ones((10,512)))
lif_dest_2 = LIF(shape=(10,))        
# Connect the OutPort of lif_src to the InPort of dense.
lif_src.s_out.connect(dense.s_in)
dense.a_out.connect(lif_dest.a_in)
lif_dest.s_out.connect(dense1.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense1.a_out.connect(lif_dest_1.a_in)
lif_dest_1.s_out.connect(dense2.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense2.a_out.connect(lif_dest_2.a_in)

### Configure profiling tools

The profiler needs as initialization the run configuration as parameter.

In [41]:
num_steps = 3
run_config = Loihi2HwCfg()
profiler = Profiler.init(run_config)

Afterwards we configure an execution time probe. Configuring an energy, activity and memory probe additionaly or extra also works.

In [42]:
profiler.execution_time_probe(num_steps=num_steps)

### Run the network

The network will run for 100 time steps.

In [43]:
# Execute Process lif_src and all Processes connected to it (dense, lif_dest). 
lif_src.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
#print("Sent spikes:", lif_src.s_out, "\nReceived spikes:", lif_dest_1.s_out)
#lif_src.stop()

Per core distribution:
----------------------------------------------------------------
| AxonIn |NeuronGr| Neurons|Synapses| AxonMap| AxonMem|  Cores |
|--------------------------------------------------------------|
|     512|       1|      10|    1536|      10|       0|       1|
|     512|       1|      73|    7899|      73|       0|       7|
|     648|       1|      51|    7128|      51|       0|      10|
|--------------------------------------------------------------|
| Total                                               |      18|
----------------------------------------------------------------


NotImplementedError: No support for (source, destination) pairs of type '(PySpikeModelFixed, NcModelDense)' yet.

In [ ]:
lif_src.stop()

## Version 3 (runs forever)

In [ ]:
inputs = []
labels = np.zeros(len(spike_dataset.samples))
for i in range(3):
    input, label = spike_dataset[i]
    inputs.append(input)
    labels[i] = label

labels = labels.tolist()

In [ ]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort
from lava.magma.core.model.sub.model import AbstractSubProcessModel
from lava.magma.core.model.py.model import PyLoihiProcessModel

# Import ProcessModel ports, data-types
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType

# Import execution protocol and hardware resources
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.resources import CPU

# Import decorators
from lava.magma.core.decorator import implements, requires
import logging
import numpy as np
import matplotlib.pyplot as plt

from lava.magma.core.run_configs import Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.magma.compiler.subcompilers.nc.ncproc_compiler import CompilerOptions
from lava.utils.profiler import Profiler


In [ ]:

class OutputProcess(AbstractProcess):
    def __init__(self, num_images):
        super().__init__()
        shape = (10,)
        
        # Creating Vars, InPorts and OutPorts (each process has these!)
        
        self.spikes_in = InPort(shape=shape)
        self.label_in = InPort(shape=(1,))
        
        self.num_images = Var(shape=(1,), init=num_images)
        # Place for acculumating spikes over the time period
        self.spikes_accum = Var(shape=shape)
        # Each image has 300 timepoints in the training data
        self.num_steps_per_image = Var(shape=(1,), init=300)
        self.pred_labels = Var(shape=(num_images,))
        # Labels for the ground truth
        self.gt_labels = Var(shape=(num_images,))

# Feeds the input to the model.

class InputProcess(AbstractProcess):
    def __init__(self, num_images, num_steps_per_image):
        super().__init__()
        shape = (648,)

        # OutPorts
        self.spikes_out = OutPort(shape=shape)
        self.label_out = OutPort(shape=(1,))

        # Vars
        self.num_images = Var(shape=(1,), init=num_images)
        self.num_steps_per_image = Var(shape=(1,), init=num_steps_per_image)
        self.input_img = Var(shape=shape+(300,))
        self.ground_truth_label = Var(shape=(1,))
        

In [ ]:
# Output ProcessModel

@implements(proc=OutputProcess, protocol=LoihiProtocol)
@requires(CPU)
class PyOutputProcessModel(PyLoihiProcessModel):
    label_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int, precision=32)
    spikes_in: PyInPort = LavaPyType(PyInPort.VEC_DENSE, bool, precision=1)
    num_images: int = LavaPyType(int, int, precision=32)
    spikes_accum: np.ndarray = LavaPyType(np.ndarray, np.int32, precision=32)
    num_steps_per_image: int = LavaPyType(int, int, precision=32)
    pred_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    gt_labels: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.current_img_id = 0 # Used to iterate through examples
        
    def post_guard(self):
        ''' Used during PostManagement, determines if an image
            has just finished being passed through'''
        return self.time_step % self.num_steps_per_image == 0 and \
                self.time_step > 1

    def run_post_mgmt(self):
        ''' Function executed after post_guard returns True'''
        # Storing prediction and ground truths
        gt_label = self.label_in.recv() 
        pred_label = np.argmax(self.spikes_accum)
        self.gt_labels[self.current_img_id] = gt_label # Indexing into Process Vars
        self.pred_labels[self.current_img_id] = pred_label

        # Setting up for next image
        self.current_img_id += 1
        self.spikes_accum = np.zeros_like(self.spikes_accum)

    def run_spk(self):
        ''' Runs at every timepoint; getting spikes from the forward pass
            and accumulating'''
        spk_in = self.spikes_in.recv()
        self.spikes_accum = self.spikes_accum + spk_in

# Input ProcessModel

@implements(proc=InputProcess, protocol=LoihiProtocol)
@requires(CPU)
class PySpikeInputModel(PyLoihiProcessModel):
    num_images: int = LavaPyType(int, int, precision=32)
    spikes_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, bool, precision=1)
    label_out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, np.int32,
                                      precision=32)
    num_steps_per_image: int = LavaPyType(int, int, precision=32)
    input_img: np.ndarray = LavaPyType(np.ndarray, int, precision=32)
    ground_truth_label: int = LavaPyType(int, int, precision=32)
    
    def __init__(self, proc_params):
        super().__init__(proc_params=proc_params)
        self.input_data = mnist_inputs
        self.gt_labels = mnist_labels
        self.curr_img_id = 0
        self.curr_img_time = 0

    def post_guard(self):
        ''' PostManagement phase guard'''
        return self.time_step % self.num_steps_per_image == 1

    def run_post_mgmt(self):
        ''' Executed when post_guard returns True, i.e. after image 
            has finished being fed through. '''
        # Getting the next image and ground truth 
        # to feed through
        self.input_img = self.input_data[self.curr_img_id].astype(np.int32)
        self.ground_truth_label = self.gt_labels[self.curr_img_id]
        
        self.label_out.send(np.array([self.ground_truth_label]))
        self.curr_img_id += 1

    def run_spk(self):
        ''' Spiking phase, executed at every timepoint '''
        # Input dataset already represents spike trains, so we just
        # send the next timepoint
        print(self.curr_img_time)
        self.spikes_out.send(self.input_img[:,self.curr_img_time])
        self.curr_img_time += 1
        

In [ ]:
lif_src = InputProcess(num_images, num_steps_per_image)
dense = Dense(weights=np.ones((512, 648)))
lif_dest = LIF(shape=(512,))
dense1 = Dense(weights=np.ones((512, 512)))
lif_dest_1 = LIF(shape=(512,))   
dense2 = Dense(weights=np.ones((10,512)))
lif_dest_2 =  OutputProcess(num_images)       
# Connect the OutPort of lif_src to the InPort of dense.
lif_src.spikes_out.connect(dense.s_in)
dense.a_out.connect(lif_dest.a_in)
lif_dest.s_out.connect(dense1.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense1.a_out.connect(lif_dest_1.a_in)
lif_dest_1.s_out.connect(dense2.s_in)
# Connect the OutPort of dense to the InPort of lif_dst.
dense2.a_out.connect(lif_dest_2.spikes_in)
lif_src.label_out.connect(lif_dest_2.label_in)    

In [ ]:
# Running on one test image
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi2SimCfg


slayer_network.run(
    condition=RunSteps(num_steps=num_steps_per_image),
    run_cfg=Loihi2SimCfg())

ground_truth = output_process.gt_labels.get().astype(np.int32)
predictions = output_process.pred_labels.get().astype(np.int32)

slayer_network.stop()
    

### Retrieve measurement results


In [ ]:
profiler.execution_time[:21]  # Time series of the execution time per time step in seconds.

In [ ]:
print(f"Total execution time: {np.round(np.sum(profiler.execution_time), 6)} s")

In [ ]:
profiler.plot_execution_time()

In [ ]:
print(f"Execution time series of spiking phase: {profiler.spiking_time[:5]}")
print(f"Execution time series of pre-learning management phase: {profiler.pre_lrn_mgtm_time[:5]}")
print(f"Execution time series of learning phase: {profiler.learning_time[:5]}")
print(f"Execution time series of  management phase: {profiler.management_time[95:]}")
print(f"Execution time series of host phase: {profiler.host_time[95:]}")

### Energy, activity and memory probe

In [11]:
num_steps = 10
run_config = Loihi2HwCfg()
profiler = Profiler.init(run_config)

In [12]:
profiler.energy_probe(num_steps=num_steps)
profiler.activity_probe()
profiler.memory_probe()

In [ ]:
# Execute Process lif_src and all Processes connected to it (dense, lif_dest).
lif_src.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
lif_src.stop()

In [ ]:
print(f"Total execution time: {np.round(np.sum(profiler.execution_time), 6)} s")
print(f"Total power: {np.round(profiler.power, 6)} W") 
print(f"Total energy: {np.round(profiler.energy, 6)} J")
print(f"Static energy: {np.round(profiler.static_energy, 6)} J") 

In [ ]:
profiler.power_breakdown()

In [ ]:
profiler.energy_breakdown()

In [ ]:
profiler.plot_activity()

In [ ]:
profiler.plot_memory_util()

In [19]:
def run_simple_lif_with_execution_time_probe(num_steps, t_start=1, t_end=num_steps, buffer_size=2048, dt=1):
    # A very simple network of 100 LIF neurons
    lif = LIF(shape=(100,), bias_mant=50, bias_exp=6, vth=100)
    
    run_config = Loihi2HwCfg()
    profiler = Profiler.init(run_config)

    # Configure an execution time probe
    profiler.execution_time_probe(num_steps=num_steps, t_start=t_start, t_end=t_end, buffer_size=buffer_size, dt=dt)

    # Run the network
    CompilerOptions.verbose = False  # Limited information output from the compiler for presentation
    lif.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
    lif.stop()

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=100)
profiler.plot_execution_time()

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=3000)
profiler.plot_execution_time()

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=100, buffer_size=64)
profiler.plot_execution_time()

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=3000, buffer_size=3500)

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=3000, buffer_size=1024, dt=3)
profiler.plot_execution_time()

In [ ]:
run_simple_lif_with_execution_time_probe(num_steps=3000, t_start=2500, t_end=2600, buffer_size=1024, dt=1)
profiler.plot_execution_time()

In [26]:
def run_simple_lif_with_energy_probe(num_steps, buffer_size=None):
    # A very simple network of 100 LIF neurons
    lif = LIF(shape=(100,), bias_mant=50, bias_exp=6, vth=1000)
    
    run_config = Loihi2HwCfg()
    profiler = Profiler.init(run_config)

    if buffer_size is not None:
        # Configure an execution time probe
        profiler.execution_time_probe(num_steps=num_steps, t_start=1, t_end=num_steps, buffer_size=buffer_size, dt=1)

    # Configure an energy probe
    profiler.energy_probe(num_steps=num_steps)

    # Run the network
    CompilerOptions.verbose = False  # Limited information output from the compiler for presentation
    lif.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
    lif.stop()
    
    return profiler

In [ ]:
profiler = run_simple_lif_with_energy_probe(num_steps=100)
print(f"Total energy: {profiler.energy} J")